# XI.9.3.a
Построить алгоритм метода пристрелки для вычисления
решения следующих нелинейных задач:\
$
\begin{cases}
   y^{''} - x\sqrt{y} = 0, 0 \leq x \leq 1 \\
   y(0) = 0, y(1) = 2
\end{cases}
$ \
Перейдем к задаче Коши:\
$
\begin{cases}
   y^{'} = \upsilon, 0 \leq x \leq 1 \\
   \upsilon' = x\sqrt{y} \\
   y(0) = 0 \\
   \upsilon(0) = \alpha
\end{cases}
$ \
Переобозначим $x \rightarrow t; y \rightarrow x; \upsilon \rightarrow y$\
$
\begin{cases}
   x' = y, 0 \leq t \leq 1 \\
   y' = t\sqrt{x} \\
   x(0) = 0 \\
   y(0) = \alpha
\end{cases}
$ \
Решаем систему методом Рунге-Кутты (классическая схема 4 порядка).  Подбор параметра $\alpha$ производим методом половинного деления

In [1]:
from math import sqrt
import numpy as np
import pandas as pd

In [2]:
def f(t, x, y):
    return y

def g(t, x, y):
    return t * sqrt(x)

In [3]:
def ODE(t_grid, tau, x_0, y_0):
    x_grid = [x_0]
    y_grid = [y_0]
    for i in range(len(t_grid) - 1):
        k1 = f(t_grid[i], x_grid[i], y_grid[i])
        m1 = g(t_grid[i], x_grid[i], y_grid[i])
        k2 = f(t_grid[i] + tau / 2, x_grid[i] + tau / 2 * k1, y_grid[i] + tau / 2 * m1)
        m2 = g(t_grid[i] + tau / 2, x_grid[i] + tau / 2 * k1, y_grid[i] + tau / 2 * m1)
        k3 = f(t_grid[i] + tau / 2, x_grid[i] + tau / 2 * k2, y_grid[i] + tau / 2 * m2)
        m3 = g(t_grid[i] + tau / 2, x_grid[i] + tau / 2 * k2, y_grid[i] + tau / 2 * m2)
        k4 = f(t_grid[i] + tau, x_grid[i] + tau * k3, y_grid[i] + tau * m3)
        m4 = g(t_grid[i] + tau, x_grid[i] + tau * k3, y_grid[i] + tau * m3)
        x_n1 = x_grid[i] + tau / 6 * (k1 + 2 * k2 + 2 * k3 + k4)
        y_n1 = y_grid[i] + tau / 6 * (m1 + 2 * m2 + 2 * m3 + m4)
        x_grid.append(x_n1)
        y_grid.append(y_n1)
    return x_grid, y_grid

In [4]:
t_min = 0
t_max = 1
x_0 = 0
x_1_target = 2

N = 1000 + 1
t_grid, tau = np.linspace(t_min, t_max, N, retstep=True)

alpha_min = 1  # x < 2
alpha_max = 2  # x > 2
alpha_table = [alpha_min, alpha_max]
x_1_table = [ODE(t_grid, tau, x_0, alpha_min)[0][-1], ODE(t_grid, tau, x_0, alpha_max)[0][-1]]
for i in range(20):
    alpha = (alpha_min + alpha_max) / 2
    x_1, y_1 = ODE(t_grid, tau, x_0, alpha)
    alpha_table.append(alpha)
    x_1_table.append(x_1[-1])
    if x_1[-1] < x_1_target:
        alpha_min = alpha
    else:
        alpha_max = alpha
x_1_delta_table = [i - x_1_target for i in x_1_table]
pd.DataFrame({"alpha": alpha_table, "x*(1)": x_1_table, "x*(1) - x(1)": x_1_delta_table})

,alpha,x*(1),x*(1) - x(1)
0,1.000000,1.116180,-8.838199e-01
1,2.000000,2.163522,1.635218e-01
2,1.500000,1.641867,-3.581329e-01
3,1.750000,1.903083,-9.691735e-02
4,1.875000,2.033389,3.338929e-02
5,1.812500,1.968259,-3.174115e-02
6,1.843750,2.000830,8.296425e-04
7,1.828125,1.984546,-1.545434e-02
8,1.835938,1.992688,-7.311999e-03
9,1.839844,1.996759,-3.241091e-03
